<a href="https://colab.research.google.com/github/niteshctrl/self_driving_car/blob/main/Self_Driving_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connecting gDrive

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from keras.layers import Conv2D, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Download Data
 
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B" -O driving.zip && rm -rf /tmp/cookies.txt

--2021-05-28 07:27:25--  https://docs.google.com/uc?export=download&confirm=LRYg&id=1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B
Resolving docs.google.com (docs.google.com)... 173.194.216.100, 173.194.216.101, 173.194.216.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.216.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-64-docs.googleusercontent.com/docs/securesc/ovi3q08ue1bhb3bu2l9qejbihl5ela9v/5pbvo4dp8dq3mte573nb68kp50h9u3n4/1622186775000/15290327230174755112/07000354846118205129Z/1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B?e=download [following]
--2021-05-28 07:27:25--  https://doc-10-64-docs.googleusercontent.com/docs/securesc/ovi3q08ue1bhb3bu2l9qejbihl5ela9v/5pbvo4dp8dq3mte573nb68kp50h9u3n4/1622186775000/15290327230174755112/07000354846118205129Z/1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B?e=download
Resolving doc-10-64-docs.googleusercontent.com (doc-10-64-docs.googleusercontent.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Conne

In [ ]:
# UNZIP Downloaded Data
 
!unzip driving.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/53174.jpg          
  inflating: data/33858.jpg          
  inflating: data/15121.jpg          
  inflating: data/8755.jpg           
  inflating: data/44408.jpg          
  inflating: data/7466.jpg           
  inflating: data/23493.jpg          
  inflating: data/12896.jpg          
  inflating: data/45716.jpg          
  inflating: data/6778.jpg           
  inflating: data/33680.jpg          
  inflating: data/1017.jpg           
  inflating: data/42079.jpg          
  inflating: data/22955.jpg          
  inflating: data/583.jpg            
  inflating: data/55505.jpg          
  inflating: data/13550.jpg          
  inflating: data/1771.jpg           
  inflating: data/55263.jpg          
  inflating: data/13236.jpg          
  inflating: data/14559.jpg          
  inflating: data/34689.jpg          
  inflating: data/45070.jpg          
  inflating: data/53612.jpg          
  inflating: data/15647.jpg          

# Problem Statement

# Import Libraries and Data

In [ ]:
df = pd.read_csv('data.txt', parse_dates=True, header=None, names=['image', 'time'])

df = df['image'].str.split(expand=True)
df.columns = ['filename','angle']

df.angle = np.asarray(df.angle).astype('float32')

# Data Analysis

In [ ]:
counter = 0
for filename in os.listdir(data_directory):
    counter += 1
print("Number of Images = {}".format(counter))

Number of Images = 63825


# Data Preparation and Preprocessing

# Data Augmentation and Flow from Directory

In [ ]:
# TBD = Regression Problem

datagen = ImageDataGenerator(rescale=1./255)


iterator = datagen.flow_from_dataframe(dataframe = df,
                                       directory='data',
                                       x_col='filename',
                                       y_col='angle', 
                                       target_size=(256,455),
                                       class_mode='raw',
                                       batch_size=64
                                       )

Found 63825 validated image filenames.


# Modelling

* Mean Model
* Layerwise Visualization

In [ ]:
model_cnn = tf.keras.Sequential([
                Conv2D(3, (5,5), activation=tf.nn.relu, input_shape=(256,455,3)),
                Conv2D(24, (5,5), activation=tf.nn.relu, kernel_initializer="he_normal"),
                Conv2D(36, (5,5), activation=tf.nn.relu, kernel_initializer="he_normal"),
                Conv2D(48, (3,3), activation=tf.nn.relu, kernel_initializer="he_normal"),
                Conv2D(64, (3,3), activation=tf.nn.relu, kernel_initializer="he_normal"),
                Flatten(),
                Dense(50, activation='relu'),
                Dense(10, activation='relu'),
                Dense(1, activation='linear')
])

model_cnn.compile(loss="mse", optimizer='adam')

In [ ]:
model_cnn.build(input_shape=(None, 256,455,3))

In [ ]:
model_cnn.summary()

In [ ]:
history = model_cnn.fit(iterator, epochs=5)

Epoch 1/5


In [ ]:
base_model = VGG16(include_top=False,
                   weights='imagenet',
                   input_shape=(455,256,3)
                   )

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
head_model = base_model.output
head_model = Flatten()(head_model)
head_model = Dense(128, activation='relu')(head_model)
head_model = Dense(64, activation='relu')(head_model)
head_model = Dense(1, activation='linear')(head_model)

# Transfer Learning

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from keras import Model

In [ ]:

vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
from keras import Sequential
from keras.layers.experimental.preprocessing import Rescaling

In [ ]:
base_model = VGG16(input_shape=(455,256,3),
                   weights='imagenet',
                   include_top=False
                   )
base_model.trainable = False

In [ ]:
model = Sequential()
model.add(Rescaling(1./255, input_shape=(455,256,3)))
model.add(base_model)
model.add(Flatten())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 134,264,641
Trainable params: 134,264,641
Non-trainable params: 0
_________________________________________________________________


# Evaluation

# Results and Conclusion

# References

[1] Mohammed AL-Ma'amari.   Deep Neural Networks for Regression Problems. URL: https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

[2] Sully Chen. How a High School Junior Made a Self-Driving Car. URL: https://towardsdatascience.com/how-a-high-school-junior-made-a-self-driving-car-705fa9b6e860

[3] Jason Brownlee. How to Load Large Datasets From Directories for Deep Learning in Keras. URL: https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/

[4] Image data preprocessing. URL: https://keras.io/api/preprocessing/image/
